## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-04-22-50-15 +0000,nyt,Judges Openly Doubt Government as Justice Dept...,https://www.nytimes.com/2025/08/04/us/politics...
1,2025-08-04-22-46-59 +0000,wapo,VOA director fired after declining reassignmen...,https://www.washingtonpost.com/business/2025/0...
2,2025-08-04-22-46-07 +0000,nyt,The Origins of the Political Power Grab in Texas,https://www.nytimes.com/2025/08/04/us/politics...
3,2025-08-04-22-42-41 +0000,nyt,Man Who Killed 4 in Arkansas Grocery Store Sho...,https://www.nytimes.com/2025/08/04/us/travis-p...
4,2025-08-04-22-40-46 +0000,startribune,Where does the future of the Twins stand follo...,https://www.startribune.com/twins-sale-pohlads...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
112,trump,40
187,new,21
347,will,12
20,texas,11
128,ex,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
256,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...,98
245,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,90
136,2025-08-04-16-50-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...,78
191,2025-08-04-13-01-22 +0000,bbc,Hundreds of Israeli ex-officials appeal to Tru...,https://www.bbc.com/news/articles/crkznje8nz8o...,78
201,2025-08-04-11-47-24 +0000,nyt,Trump to Appoint New Top Labor Statistics Offi...,https://www.nytimes.com/2025/08/03/us/politics...,76


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
256,98,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...
223,57,2025-08-04-09-03-26 +0000,nyt,Texas Redistricting Fight Prompts Democratic G...,https://www.nytimes.com/2025/08/04/us/politics...
33,52,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
159,47,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
48,46,2025-08-04-21-20-09 +0000,nypost,NYC’s iconic Flatiron Building will get bathed...,https://nypost.com/2025/08/04/business/nycs-ic...
55,41,2025-08-04-21-04-00 +0000,wsj,The State Department may require travelers ent...,https://www.wsj.com/politics/policy/trump-admi...
122,40,2025-08-04-17-36-12 +0000,nypost,Boeing fighter jet workers go on strike after ...,https://nypost.com/2025/08/04/business/boeing-...
42,36,2025-08-04-21-32-27 +0000,nypost,"Girl, 1, found dead in apparent drowning at NY...",https://nypost.com/2025/08/04/us-news/girl-1-f...
90,35,2025-08-04-19-31-32 +0000,nypost,Trump lambasts ‘Russian war machine’ as specia...,https://nypost.com/2025/08/04/world-news/trump...
104,34,2025-08-04-18-44-19 +0000,nypost,Netanyahu approves plan to fully occupy Gaza S...,https://nypost.com/2025/08/04/world-news/netan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
